In [1]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
dataset=pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv')
dataset.head()

2024-05-23 11:35:35.194335: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 11:35:35.194453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 11:35:35.332530: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,en,fr
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...
1,Site map,Plan du site
2,Feedback,Rétroaction
3,Credits,Crédits
4,Français,English


In [2]:
len(dataset['en'][1])

8

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')
print("[ BEFORE ] tokenizer vocab size:", len(tokenizer))
def embedder(text):
    global tokenizer,model
    input_ids = tokenizer(text, return_tensors='tf')
    arr=text.split(" ")
    num_added_toks = tokenizer.add_tokens(arr)
    model.resize_token_embeddings(len(tokenizer))
    outputs = model(input_ids)
    #Applying Padding
    if outputs[0].shape[1]>100:
        cls_embedding=outputs[0][:,:100,:]
    else:
        cls_embedding=tf.concat([outputs[0], tf.zeros([1,100-outputs[0].shape[1],768])], axis=1)
    #Positional encoding 
    seq_len,d,n=100,768,10000
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    #Adding positional encoding
    cls_embedding += [P]
    return cls_embedding
    

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

[ BEFORE ] tokenizer vocab size: 105879


In [4]:
embedder("Some weights of the PyTorch model")

<tf.Tensor: shape=(1, 100, 768), dtype=float32, numpy=
array([[[-0.14002718,  0.99718755, -0.02695617, ...,  0.97366893,
          0.05828621,  0.9977569 ],
        [ 0.6059175 ,  0.51176375,  0.79083943, ...,  0.8594985 ,
          0.66855276,  0.7021471 ],
        [ 0.42150906, -0.32307467,  1.172154  , ...,  0.7035326 ,
          0.40775174,  0.43801546],
        ...,
        [ 0.37960774, -0.92514753,  0.4379616 , ...,  0.9999482 ,
          0.00993531,  0.99995065],
        [-0.5733819 , -0.81928825,  0.9900526 , ...,  0.99994713,
          0.01003773,  0.99994963],
        [-0.99920684,  0.03982088,  0.67107844, ...,  0.99994606,
          0.01014015,  0.99994856]]], dtype=float32)>

In [5]:
import math
def encoder(embed):
    # weights of query ,keys and values and other weights
    Wq=tf.random.normal(shape=(1,100, 768))
    Wk=tf.random.normal(shape=(1,100, 768))
    Wv=tf.random.normal(shape=(1,100, 768))
    W0=tf.random.normal(shape=(1,100, 768))
    W1=tf.random.normal(shape=(1,100, 768))
    
    #Calculating query ,keys and values
    Query=embed*Wq
    Key=embed*Wk
    Value=embed*Wv
    # Scaled-Dot Product Attention
    scores = tf.matmul(Query, Key, transpose_b=True) / math.sqrt(tf.cast(768, tf.float32))
    weights = tf.keras.activations.softmax(scores)
    result=tf.matmul(weights, Value)
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    #feed forward layer from scratch
    Layer1=W0*normalized_embeddings
    result1=tf.nn.relu(Layer1)
    result=W1*normalized_embeddings
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    Key=normalized_embeddings*Wk
    Value=normalized_embeddings*Wv
    return [Key,Value]

In [6]:
encoder(embedder("Some weights of the PyTorch model"))

[<tf.Tensor: shape=(1, 100, 768), dtype=float32, numpy=
 array([[[-6.9696262e-02,  1.3283433e-01,  4.9304453e-01, ...,
          -7.0737416e-01,  2.9943779e-01, -3.4968057e-01],
         [ 6.1217606e-01,  1.5933023e-01, -2.2136996e+00, ...,
           6.4097950e-03,  8.4499987e-03, -3.1447878e-01],
         [ 2.1455662e-01,  1.1204567e-01, -1.4696792e+00, ...,
           3.2027364e-01, -1.5418515e-02, -3.4932010e-02],
         ...,
         [ 7.3659448e-03,  5.0475317e-01, -4.0500172e-04, ...,
          -6.8118066e-01, -1.2738823e-02, -8.0211443e-01],
         [ 6.1150946e-02, -1.0367657e+00,  5.0811577e+00, ...,
           3.5533053e-01, -5.4936871e-02,  1.4436677e-01],
         [-2.5086007e-01,  2.3103105e-03,  1.3020708e-02, ...,
           4.6312943e-01,  2.1066266e-01,  3.7164634e-01]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 100, 768), dtype=float32, numpy=
 array([[[-1.51138857e-01,  1.54459970e-02, -2.33535558e-01, ...,
          -5.15068769e-01, -1.90796995e+00, -8.07234347e-

In [7]:
def decoder(embed, arr):
    # weights of query ,keys and values and other weights
    Wq=tf.random.normal(shape=(1,100, 768))
    Wk=tf.random.normal(shape=(1,100, 768))
    Wv=tf.random.normal(shape=(1,100 ,768))
    W0=tf.random.normal(shape=(1,100 ,768))
    W1=tf.random.normal(shape=(1,100 ,768))
    W01=tf.random.normal(shape=(1,100, 768))
    W11=tf.random.normal(shape=(1,100, 768))
    
    #Calculating query ,keys and values
    Query=embed*Wq
    Key=embed*Wk
    Value=embed*Wv
    # Scaled-Dot Product Attention
    scores = tf.matmul(Query, Key, transpose_b=True) / math.sqrt(tf.cast(768, tf.float32))
    weights = tf.keras.activations.softmax(scores)
    result=tf.matmul(weights, Value)
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    #feed forward layer from scratch
    Layer1=W0*normalized_embeddings
    result1=tf.nn.relu(Layer1)
    result=W1*normalized_embeddings
    #Add layer
    embed+=result
    #normalize the embeddings
    embed = tf.keras.layers.LayerNormalization()(embed)
    #key ,value of encodings output
    Key,Value=arr
    Query=embed*Wq
    scores = tf.matmul(Query, Key, transpose_b=True) / math.sqrt(tf.cast(768, tf.float32))
    weights = tf.keras.activations.softmax(scores)
    result=tf.matmul(weights, Value)
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    #feed forward layer from scratch
    Layer1=W01*normalized_embeddings
    result1=tf.nn.relu(Layer1)
    result=W11*normalized_embeddings
    #Add layer
    embed+=result
    return embed

In [8]:
print(decoder(embedder(dataset['fr'][1]),encoder(embedder(dataset['en'][1]))))

tf.Tensor(
[[[-0.12480693  1.2203567  -0.01914754 ...  0.6471748  -0.39092898
    1.0175033 ]
  [-0.0875829   1.6260924   0.8490184  ...  0.8626851  -1.0175185
   -0.64678293]
  [ 0.18277653  0.9410808  -0.0764506  ...  2.764442   -0.01304665
   -0.12756665]
  ...
  [ 0.13781004 -0.01139675 -0.07149631 ...  0.42931557 -0.79244065
    0.25977308]
  [ 0.16284919  3.586234   -0.5499325  ... -0.55206466 -0.13128331
   -0.8038197 ]
  [-1.0139439   0.11074829 -1.5799692  ...  0.7790674   0.4080514
    0.1137085 ]]], shape=(1, 100, 768), dtype=float32)


In [9]:
def train(input1,output1):
    global model,tokenizer
    W0=tf.random.normal(shape=(1,100, 768))
    encoder_output=encoder(embedder(input1))
    output_embed=embedder(output1)
    decoder_output=decoder(output_embed,encoder_output)
    Layer1=W0*decoder_output
    result=tf.nn.softmax(Layer1, axis=-1)
    predicted_token_ids = tf.argmax(result, axis=-1).numpy().flatten()
    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_ids)
    predicted_text = tokenizer.convert_tokens_to_string(predicted_tokens)
    return predicted_text

In [10]:
print(train(dataset['fr'][1],dataset['en'][1]))

؍ ϕ ب ر [unused21] [unused65] જ [unused97] ઊ [unused68] [unused58] ઞ [unused13] ল ੦ [unused24] 0 8 6 : ઃ 9 [unused69] _ ։ մ [unused52] ɒ պ [unused20] [unused3] § z [unused67] § [unused78] þ ¦ ¯ x ¾ ੦ / 8 ʁ פ ß { ɣ ʳ ۳ ل [unused92] খ [unused75] ƒ ¿ g ɤ ध ʃ [unused11] » श ø ι ɲ [unused68] - u অ t ɦ [unused19] ν ब [unused13] [unused87] ঞ 7 झ λ ट ई ɨ [unused88] [ ਨ « " ی ʔ र ʋ [unused49] ρ श ՝ _ [unused24]
